# Splane&Scube tutorial (1/2): Identify uniform spatial domain on human brain MERFISH dataset

July 2023

Dataset: 33 MERFISH slices of mouse brain ([here](https://zenodo.org/record/8167488))

## Data preprocessing

In [1]:
from SPACEL.setting import set_environ_seed
set_environ_seed(42)
from SPACEL import Splane
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib

Setting environment seed: 42


In [2]:
import sys
import os
sys.path.append('/Users/u0096223/Documents/GitHub/PVLabTime/Merscope')
data_dir = '/Users/u0096223/Baidu/Science/Data PVDH/MERSCOPE data/P5_mouse_brain_CBD_panel/Cells'
# anndata_filename = os.path.join(data_dir,'15-11-24_23-22_P5_mouse_brain_CBD_panel_Cells_annotated.h5ad')
# anndata_filename = os.path.join(data_dir,'Spoint_result_mouse_brain_st.h5ad')

fig_root = os.path.join(data_dir,'Figures_spatial_mapping_retroSeq_xu')

# sc_files ='/Users/u0096223/Documents/Transcriptomics/P5_thalamocortical_240715/P5_thalamocortical_Xcount.h5ad'
spatial_file =os.path.join(fig_root,'Spoint_result_mouse_brain_st.h5ad')
spatial_file =os.path.join(data_dir,'15-11-24_23-22_P5_mouse_brain_CBD_panel_Cells_annotated.h5ad')


# pn_trained_model= os.path.join(fig_root,'SPOINT_trained_model.h5')


In [3]:

st_merfish = sc.read_h5ad(spatial_file)


In [4]:
st_merfish.obs

,Custom Cells groups,Datasets,volume,center_x,center_y,cell_id,class_label,class_name,class_correlation_coefficient,subclass_label,subclass_name,subclass_correlation_coefficient,supertype_label,supertype_name,supertype_correlation_coefficient,cluster_label,cluster_name,cluster_alias,cluster_correlation_coefficient,leiden
4176488300005100074,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,918.626099,3135.893269,407.215646,4176488300005100074,CS20230722_CLAS_33,33 Vascular,0.5529,CS20230722_SUBC_329,329 ABC NN,0.5529,CS20230722_SUPT_1186,1186 ABC NN_1,0.5529,CS20230722_CLUS_5294,5294 ABC NN_1,5257,0.5529,9
4176488300005100077,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,1296.579224,3093.065556,416.124569,4176488300005100077,CS20230722_CLAS_14,14 HY Glut,0.3800,CS20230722_SUBC_129,129 VMH Nr5a1 Glut,0.3800,CS20230722_SUPT_0566,0566 VMH Nr5a1 Glut_1,0.3800,CS20230722_CLUS_2304,2304 VMH Nr5a1 Glut_1,2086,0.3800,21
4176488300005100078,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,1098.988892,3140.836221,414.966652,4176488300005100078,CS20230722_CLAS_33,33 Vascular,0.5413,CS20230722_SUBC_330,330 VLMC NN,0.5413,CS20230722_SUPT_1187,1187 VLMC NN_1,0.5413,CS20230722_CLUS_5296,5296 VLMC NN_1,5259,0.5413,9
4176488300005100079,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,1901.925781,3147.923169,418.713692,4176488300005100079,CS20230722_CLAS_33,33 Vascular,0.6124,CS20230722_SUBC_330,330 VLMC NN,0.6124,CS20230722_SUPT_1187,1187 VLMC NN_1,0.6124,CS20230722_CLUS_5296,5296 VLMC NN_1,5259,0.6124,9
4176488300005100084,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,825.530334,3150.803025,427.134870,4176488300005100084,CS20230722_CLAS_33,33 Vascular,0.4082,CS20230722_SUBC_330,330 VLMC NN,0.4082,CS20230722_SUPT_1187,1187 VLMC NN_1,0.4082,CS20230722_CLUS_5296,5296 VLMC NN_1,5259,0.4082,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176488700194200069,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,1077.251953,4224.746444,12715.737318,4176488700194200069,CS20230722_CLAS_33,33 Vascular,0.4778,CS20230722_SUBC_330,330 VLMC NN,0.4778,CS20230722_SUPT_1188,1188 VLMC NN_2,0.4778,CS20230722_CLUS_5300,5300 VLMC NN_2,5262,0.4778,9
4176488700194200077,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,879.106812,4137.425501,12724.852494,4176488700194200077,CS20230722_CLAS_30,30 Astro-Epen,0.2980,CS20230722_SUBC_323,323 Ependymal NN,0.2980,CS20230722_SUPT_1176,1176 Ependymal NN_2,0.2980,CS20230722_CLUS_5261,5261 Ependymal NN_2,14877,0.2980,34
4176488700194200083,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,731.352478,4071.412604,12723.966420,4176488700194200083,CS20230722_CLAS_33,33 Vascular,0.2601,CS20230722_SUBC_333,333 Endo NN,0.2601,CS20230722_SUPT_1193,1193 Endo NN_1,0.2601,CS20230722_CLUS_5311,5311 Endo NN_1,5255,0.2601,34
4176488700194200116,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,677.103821,4096.312602,12741.254466,4176488700194200116,CS20230722_CLAS_25,25 Pineal Glut,0.3208,CS20230722_SUBC_262,262 Pineal Crx Glut,0.3208,CS20230722_SUPT_1030,1030 Pineal Crx Glut_1,0.3208,CS20230722_CLUS_4606,4606 Pineal Crx Glut_1,5149,0.3208,34


Here, we will incorporate the cell type composition predicted by **Spoint** into the spatial anndata object for subsequent spatial domain identification in **Splane** using the `add_cell_type_composition` function. This function takes a DataFrame containing the cell type composition matrix as input for spot-based spatial transcriptomic data or a series of cell type annotations as input for single-cell resolution spatial transcriptomic data.

In [5]:
Splane.utils.add_cell_type_composition(st_merfish, celltype_anno=st_merfish.obs['supertype_name'])
# adata_list = Splane.utils.split_ad(st_merfish,'slice_id')
adata_list = st_merfish[st_merfish.obs['Datasets'].str.contains('region_0')]

In [6]:
adata_list.obsm['spatial'] = pd.concat([adata_list.obs['center_x'],adata_list.obs['center_y']],axis=1)

In [ ]:
# First, let's check your data structure
for i, adata in enumerate(adata_list):
    print(f"Dataset {i} shape: {adata.shape}")
    print(f"Dataset {i} spatial coordinates present: {('center_x' in adata.obs.columns) and ('center_y' in adata.obs.columns)}")
    print(f"Number of spots: {adata.n_obs}")

# Custom function to safely initialize spatial neighbors
def safe_init_spatial_neighbors(adata, min_neighbors=1):
    """
    Safely initialize spatial neighbors considering data size
    """
    import squidpy as sq
    import numpy as np
    
    n_samples = adata.n_obs
    
    if n_samples <= 1:
        print(f"Warning: Dataset has only {n_samples} samples, skipping neighbor calculation")
        return adata
    
    # Calculate optimal number of neighbors based on data size
    optimal_n_neighs = min(min_neighbors, n_samples - 1)
    
    # Try grid-based neighbors first
    try:
        sq.gr.spatial_neighbors(adata, 
                              coord_type='grid', 
                              n_neighs=optimal_n_neighs)
    except ValueError:
        # If grid fails, try generic spatial neighbors
        try:
            sq.gr.spatial_neighbors(adata, 
                                  coord_type='generic', 
                                  n_neighs=optimal_n_neighs)
        except Exception as e:
            print(f"Warning: Could not compute spatial neighbors: {str(e)}")
            
    return adata

# Modified initialization code
def safe_init_model(adata_list, n_clusters, use_gpu=False, min_neighbors=1, **kwargs):
    """
    Safely initialize SPACEL model with proper neighbor handling
    """
    # Filter out any empty datasets
    valid_adata_list = [adata for adata in adata_list if adata.n_obs > 0]
    
    if not valid_adata_list:
        raise ValueError("No valid datasets found")
        
    # Process each dataset
    processed_adata_list = []
    for adata in valid_adata_list:
        if adata.n_obs >= min_neighbors:
            processed_adata = safe_init_spatial_neighbors(adata.copy(), min_neighbors)
            processed_adata_list.append(processed_adata)
            
    if not processed_adata_list:
        raise ValueError("No datasets remaining after processing")
    
    # Initialize model with processed data
    from SPACEL.Splane import Splane
    return Splane.init_model(processed_adata_list, 
                           n_clusters=n_clusters,
                           use_gpu=use_gpu,
                           n_neighbors=min_neighbors,
                           **kwargs)

Dataset 0 shape: (1, 498)
Dataset 0 spatial coordinates present: True
Number of spots: 1
Dataset 1 shape: (1, 498)
Dataset 1 spatial coordinates present: True
Number of spots: 1
Dataset 2 shape: (1, 498)
Dataset 2 spatial coordinates present: True
Number of spots: 1
Dataset 3 shape: (1, 498)
Dataset 3 spatial coordinates present: True
Number of spots: 1
Dataset 4 shape: (1, 498)
Dataset 4 spatial coordinates present: True
Number of spots: 1
Dataset 5 shape: (1, 498)
Dataset 5 spatial coordinates present: True
Number of spots: 1
Dataset 6 shape: (1, 498)
Dataset 6 spatial coordinates present: True
Number of spots: 1
Dataset 7 shape: (1, 498)
Dataset 7 spatial coordinates present: True
Number of spots: 1
Dataset 8 shape: (1, 498)
Dataset 8 spatial coordinates present: True
Number of spots: 1
Dataset 9 shape: (1, 498)
Dataset 9 spatial coordinates present: True
Number of spots: 1
Dataset 10 shape: (1, 498)
Dataset 10 spatial coordinates present: True
Number of spots: 1
Dataset 11 shape: (

In [ ]:
# Use the safe initialization
try:
    splane_model = safe_init_model(adata_list, 
                                 n_clusters=7,
                                 use_gpu=False,
                                 min_neighbors=1)
except Exception as e:
    print(f"Error initializing model: {str(e)}")
    
    # Print detailed diagnostics
    print("\nDetailed data diagnostics:")
    for i, adata in enumerate(adata_list):
        print(f"\nDataset {i}:")
        print(f"Shape: {adata.shape}")
        print(f"Number of spots: {adata.n_obs}")
        print(f"Spatial coordinates:")
        print(adata.obs[['center_x', 'center_y']].describe())

## Training Splane model

In this step, we initialize the Splane model by ``Splane.init_model(...)`` using the anndata object list as input. The ``n_clusters`` parameter determines the number of spatial domains to be identified. The ``k`` parameter controls the degree of neighbors considered in the model, with a larger ``k`` value resulting in more emphasis on global structure rather than local structure. The ``gnn_dropout`` parameter influences the level of smoothness in the model’s predictions, with a higher ``gnn_dropout`` value resulting in a smoother output that accommodates the sparsity of the spatial transcriptomics data. 

We train the model by ``splane.train(...)`` to obtain latent feature of each spots/cells. The parameter ``d_l`` affects the level of batch effect correction between slices. By default, ``d_l`` is ``0.2`` for spatial transcriptomics data with single cell resolution.

Then, we can identify the spatial domain to which each spot/cell belongs by ``splane.identify_spatial_domain(...)``. By default, the results will be saved in ``spatial_domain`` column in ``.obs``. If the key parameter is provided, the results will be saved in ``.obs[key]``.

In [15]:
# splane_model = Splane.init_model(adata_list, n_clusters=7,use_gpu=False,n_neighbors=1, gnn_dropout=0.5)
splane_model = Splane.init_model(adata_list, n_clusters=7,use_gpu=False,n_neighbors=1)

splane_model.train(d_l=0.2)
splane_model.identify_spatial_domain()

Setting global seed: 42


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 2

In [7]:
sc.concat(adata_list).write(f'../data/merfish_mouse_brain/merfish_mouse_brain.h5ad')